In [2]:
# run sentiment analysis for finbert (AutoModelForSequenceClassification)

from utils import *
from tqdm import tqdm
import pandas as pd
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
device = "mps"

print("----- Load Dataset -----")
# use appropriate dataset for inference
# dataset="FPB"
dataset="FIQASA"
test_dataset = TestDataset(dataset=dataset, use_template=False)
test_dataloader = DataLoader(test_dataset, batch_size=8, collate_fn=test_dataset.collate_fn)


print("----- Load Tokenizer and Model -----")
model_name_or_path = "ProsusAI/finbert"
model_name = "finbert"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, 
                                            padding_side="left",
                                            use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path,
                                                            torch_dtype=torch.bfloat16,
                                                            device_map=device)

print("----- Run Sentiment Analysis -----")
labels = []
for i, batch in tqdm(enumerate(test_dataloader)):
    tokenized_batch = tokenizer(batch['sentence'], 
                                padding=True, 
                                truncation=True, 
                                return_tensors="pt").to(device)
    output_logits = model(**tokenized_batch).logits

    label_map = {0: 'positive', 1: 'negative', 2: 'neutral'}
    output_labels = torch.argmax(output_logits, dim=-1)
    labels.extend([label_map[label.item()] for label in output_labels])

print("----- Save Outputs -----")
save_to_csv(dataset_map[dataset]['path'], model_name, labels)


----- Load Dataset -----
----- Load Tokenizer and Model -----


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


----- Run Sentiment Analysis -----


30it [00:03,  7.76it/s]

----- Save Outputs -----
Save to data/sentimental_analysis/FIQASA/test.csv
